<a href="https://colab.research.google.com/github/MWSteams/4590FSP/blob/Hydraulics/Hydraulics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hydraulics

Authors: Matthew Snyder, Ke Xu, Jonathan Harris



In [0]:
!pip install aguaclara
!pip install molmass

     |████████████████████████████████| 92kB 2.8MB/s 
     |████████████████████████████████| 163kB 8.8MB/s 
  Created wheel for aguaclara: filename=aguaclara-0.2.0-cp36-none-any.whl size=99623 sha256=4880676afe0c30e0e16ad8b964a7a96c5522c7aa7f231962cbdf9a0d25894c05
  Stored in directory: /root/.cache/pip/wheels/34/e9/34/0d3722243083a982623290c205c14a08b54ed290ee29219fd2
  Created wheel for pint: filename=Pint-0.8.1-py2.py3-none-any.whl size=134156 sha256=7600138d6c2d35c6ad84745d3773cecab810be630e2ea22cdc6afcf0a1814fac
  Stored in directory: /root/.cache/pip/wheels/b4/56/36/7c4d6a4e3660e233b95e3b0482551915cb184abca0f93ee1dd
Successfully built aguaclara pint
     |████████████████████████████████| 71kB 2.2MB/s 


In [0]:
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import molmass as mol
!wget https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
import Fairmont as fmt

--2020-04-02 15:11:36--  https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1632 (1.6K) [text/plain]
Saving to: ‘Fairmont.py’

Fairmont.py         100%[===================>]   1.59K  --.-KB/s    in 0s      

2020-04-02 15:11:36 (209 MB/s) - ‘Fairmont.py’ saved [1632/1632]



# Task Definition

Write a paragraph describing what is included in your task.

# Executive Summary 

Highlight dominant constraints and your recommendation for the design.


# Constraints

* Relevant regulations
* client expectations
* site and budget
* process capabilities including process design equations
* ease of use 
* operation requirements
* maintainence requirements

# Physics

Briefly describe the physics that will determine whether your portion of the design succeeds or fails. Present relevant equations describing those physical laws and explain how you will use those concepts to guide your design.

Replace this with equations that are important to your part of the design. Explain what each equation means.

$$ F = ma$$

where $F$ is the force, $m$ is the mass, and $a$ is the acceleration.

# Table listing potential strategies to meet the constraints and treatment objectives

Please remove my two rows of examples!

| strategy | advantages  | disadvantages | data needed for decision |
|---|---|---|---|
| physics-based design | high potential for success | requires knowledge | lots |
| wishful design | easy design steps | likely won't work | none |



# Vendors and contact information file

See this [tab delimited spreadsheet](https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt).

Add rows to the vendor spreadsheet as you find them. The sheet can easily be edited in Excel. Just make sure to save it as a tab delimited sheet. Push your changes to github so the entire class has access to the document.



In [0]:
vendors = pd.read_csv('https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt',sep='\t')
vendors.head()

,item description,vendor,contact first name,contact last name,vendor website,product info,date of contact,who made the contact,notes
0,plate settlers,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/literaturemedia/...,4/4/2020,no one,nan


# Shared assumptions

See this [python file, Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py)

Use this file to hold parameters that are in the SOW or elsewhere that
influence the design of the entire plant. This provides an easy mechanism
for all of us to share the same assumptions. Read the SOW to find sections that relate to your part of the design and enter all pertinent variables into [Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py) and then push those changes to github.

# Major Decisions

Write several paragraphs describing the major decisions that you will need to make in order to create a design and outline a strategy for how you will obtain the necessary information to make those decisions.

# Next Steps and Challenges

Summarize the next steps that you will be taking to move the design forward. Identify any challenges where you may need assistance.

# Design

Begin writing the code to start your design. Define flow rates, mass rates, energy requirements, and any readily calculated dimensions, velocities, flows, etc.

In [0]:
print('The average flow to be treated at Fairmont is',fmt.q_ave.to(u.Mgal/u.day),'or')
print('The maximum flow to be treated at Fairmont is',fmt.q_max.to(u.Mgal/u.day))
specific_energy = (fmt.electricity_power/fmt.q_ave).to(u.J/u.L)
temp = 15 * u.degC
specific_potential_energy = (specific_energy / u.gravity / ac.density_water(temp)).to(u.m)
print('The average amount of energy used per liter of water is expected to be',specific_energy,'or',specific_potential_energy,'of potential energy')

The average flow to be treated at Fairmont is 206.8 megagallon / day or
The maximum flow to be treated at Fairmont is 465.3 megagallon / day
The average amount of energy used per liter of water is expected to be 118.1 joule / liter or 12.05 meter of potential energy
